The toy model
to get frame, trajectory, put on lstm

In [1]:
import torch
import torch.nn as nn
from data_classes import *

In [4]:
e = ETH()
V = Videodata(e)
empty_frame = V.getFrame(1)

In [15]:
traj = V.read_file('/home/yy/ADL4CV/sgan/datasets/hotel/train/biwi_eth_train.txt') # all trajectories
agent_id = 11
traj_agent = traj[np.where(traj[:, 1]==agent_id)]
traj_agent.shape

(20, 4)

In [31]:
traj_agent[:,2::]

array([[  1.25100000e+01,   6.19000000e+00],
       [  1.15400000e+01,   6.03000000e+00],
       [  1.09600000e+01,   5.97000000e+00],
       [  1.02900000e+01,   6.12000000e+00],
       [  9.88000000e+00,   6.21000000e+00],
       [  9.54000000e+00,   6.09000000e+00],
       [  8.87000000e+00,   5.99000000e+00],
       [  8.04000000e+00,   5.66000000e+00],
       [  7.17000000e+00,   5.45000000e+00],
       [  6.43000000e+00,   5.23000000e+00],
       [  5.67000000e+00,   5.16000000e+00],
       [  4.94000000e+00,   4.95000000e+00],
       [  4.26000000e+00,   4.71000000e+00],
       [  3.54000000e+00,   4.54000000e+00],
       [  2.82000000e+00,   4.35000000e+00],
       [  2.16000000e+00,   4.19000000e+00],
       [  1.39000000e+00,   3.97000000e+00],
       [  7.10000000e-01,   3.72000000e+00],
       [  1.00000000e-02,   3.41000000e+00],
       [ -6.30000000e-01,   3.08000000e+00]])

In [32]:
traj_agent

array([[  1.05000000e+03,   1.10000000e+01,   1.25100000e+01,
          6.19000000e+00],
       [  1.06000000e+03,   1.10000000e+01,   1.15400000e+01,
          6.03000000e+00],
       [  1.07000000e+03,   1.10000000e+01,   1.09600000e+01,
          5.97000000e+00],
       [  1.08000000e+03,   1.10000000e+01,   1.02900000e+01,
          6.12000000e+00],
       [  1.09000000e+03,   1.10000000e+01,   9.88000000e+00,
          6.21000000e+00],
       [  1.10000000e+03,   1.10000000e+01,   9.54000000e+00,
          6.09000000e+00],
       [  1.11000000e+03,   1.10000000e+01,   8.87000000e+00,
          5.99000000e+00],
       [  1.12000000e+03,   1.10000000e+01,   8.04000000e+00,
          5.66000000e+00],
       [  1.13000000e+03,   1.10000000e+01,   7.17000000e+00,
          5.45000000e+00],
       [  1.14000000e+03,   1.10000000e+01,   6.43000000e+00,
          5.23000000e+00],
       [  1.15000000e+03,   1.10000000e+01,   5.67000000e+00,
          5.16000000e+00],
       [  1.16000000e